# 机器学习势函数计算流程简明教程

本教程展示如何调用不同的机器学习势函数（MLP）模型来进行材料性质计算。
涵盖性质：结构优化、弹性常数、声子谱、准谐近似 (QHA) 和热导率 (Kappa)。

注意：请确保已安装 `ase`, `pymatgen`, `phonopy`, `phono3py` 等依赖库。

In [2]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# 确保路径包含项目根目录，以便导入 calculators 包
# 假设当前 notebook 在 example 目录下，而 calculators 在上一级
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import numpy as np
from ase.build import bulk
from ase.io import read

# 导入计算模块
# 您可以根据需要更改导入的模型，例如 MACEModel, CHGNetModel 等
from calculators.SevenNet.sevennet_model import SevenNetModel
from calculators.MatterSim.mattersim_model import MatterSimModel
from calculators.relax_set import Relaxer
from calculators.elastic_set import ElasticSet
from calculators.phonon_set import PhononSet
from calculators.qha_set import QHASet
from calculators.kappa_set import KappaSet

print("Imports successful.")

Imports successful.


In [3]:
# 1. 初始化模型与结构
# 这里我们使用 M3GNet 作为示例，因为它通常比较容易配置
# 如果您有其他模型的权重文件，可以参照类似的接口初始化

calculator = MatterSimModel(device='cpu').calcu
# 创建一个简单的硅结构用于演示
# atoms = bulk('Si', 'diamond', a=5.43)
atoms = read('POSCAR') # 也可以从文件读取
print(f"Structure created: {atoms}")

2026-01-15 13:22:37.519 | INFO     | mattersim.forcefield.potential:from_checkpoint:893 - Loading the model from /media/kai/data/MOF/potential/mattersim/mattersim-v1.0.0-1M.pth
Structure created: Atoms(symbols='C8', pbc=True, cell=[[3.5607451090903233, 0.0, 2e-16], [6e-16, 3.5607451090903233, 2e-16], [0.0, 0.0, 3.5607451090903233]])


In [4]:
# 2. 结构优化 (Relaxation)
# 使用 BFGS 算法优化原子位置和晶胞

relaxer = Relaxer(calculator=calculator, optimizer="BFGS")

# fmax: 力收敛阈值, relax_cell: 是否优化晶胞
relaxed_atoms = relaxer.relax(structure=atoms, fmax=0.01, relax_cell=True)

print(f"Optimization finished.")
print(f"Energy: {relaxed_atoms.get_potential_energy():.4f} eV")
print(f"Lattice: {relaxed_atoms.cell.cellpar()}")

      Step     Time          Energy          fmax
BFGS:    0 13:22:43      -72.836769        1.125241
BFGS:    1 13:22:43      -72.745979        4.824952
BFGS:    2 13:22:44      -72.841660        0.050433
BFGS:    3 13:22:44      -72.841667        0.002439
Optimization finished.
Energy: -72.8417 eV
Lattice: [ 3.57107848  3.57107992  3.57107949 90.00001861 89.99996763 90.0000541 ]


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
elastic = ElasticSet(calculator=calculator, relaxer=relaxer)
elastic_dir = "calc_mattersim/elastic"
os.makedirs(elastic_dir, exist_ok=True)

structure = AseAtomsAdaptor.get_structure(relaxed_atoms)
elastic.get_elastic(calcu_dir=elastic_dir,struct=structure)
print(f"Elastic energy calculation done in '{elastic_dir}'")

      Step     Time          Energy          fmax
BFGS:    0 13:22:57      -72.833496        0.000018
      Step     Time          Energy          fmax
BFGS:    0 13:22:57      -72.809265        0.000015
      Step     Time          Energy          fmax
BFGS:    0 13:22:57      -72.769501        0.000013
      Step     Time          Energy          fmax
BFGS:    0 13:22:57      -72.714706        0.000016
      Step     Time          Energy          fmax
BFGS:    0 13:22:57      -72.833282        0.000012
      Step     Time          Energy          fmax
BFGS:    0 13:22:57      -72.807854        0.000012
      Step     Time          Energy          fmax
BFGS:    0 13:22:57      -72.764923        0.000012
      Step     Time          Energy          fmax
BFGS:    0 13:22:58      -72.703987        0.000014
      Step     Time          Energy          fmax
BFGS:    0 13:22:58      -72.841667        0.000011
      Step     Time          Energy          fmax
BFGS:    0 13:22:58      -72.827

In [6]:
# 4. 声子谱 (Phonons)
# 使用有限位移法计算声子

ph_set = PhononSet(calculator=calculator)
ph_dir = "calc_mattersim/phonon"

# dim=[2,2,2] 对应 2x2x2 超胞
has_imag = ph_set.get_phonon(
    atoms=relaxed_atoms, 
    calcu_dir=ph_dir, 
    supercell_matrix=np.array([2, 2, 2]), 
    mesh=[10, 10, 10],  # q点网格用于DOS/热力学性质
    if_thermal=True     # 计算热力学性质
)

print(f"Phonon calculation done. Has imaginary modes: {has_imag}")
print(f"Check results in '{ph_dir}'")

Supercell matrix for 2nd force constants : 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


Inferring forces for displaced atoms and computing fcs ...


100%|██████████| 1/1 [00:00<00:00,  3.48it/s]


Max drift of force constants: 0.00019246 (yy) 0.00015823 (xx) 
Max drift after traditional symmetrization: -0.00000000 (xx) -0.00000000 (xx) 
Qpoints mesh for Brillouin Zone integration : [12 12 12]
Dispersion relations using phonopy for C8 ...

ph: [[2 0 0]
 [0 2 0]
 [0 0 2]]
Has imaginary phonon: False
Thermal properties written to calc_mattersim/phonon/thermal_properties.yaml and thermal_properties.dat
Phonon calculation done. Has imaginary modes: False
Check results in 'calc_mattersim/phonon'


In [7]:
# 5. 准谐近似 (QHA)
# 计算 Gruneisen 参数和热膨胀性质
# QHA 需要在不同体积下计算声子，计算量通常是单次声子计算的 11 倍左右
qha_set = QHASet(calculator=calculator, device='cpu', dim=[2, 2, 2], mesh=[30, 30, 30], n=11, nscale=0.003, is_rm_dir=True)
qha_dir = "calc_mattersim/qha"
# 这一步会进行多次结构优化和声子计算
# dim 指定超胞大小，例如此处为 2x2x2
qha_set.get_gruneisen(
    struct=relaxed_atoms, 
    calcu_dir=qha_dir, 
)
print(f"QHA calculation finished in '{qha_dir}'")

      Step     Time          Energy          fmax
BFGS:    0 13:23:35      -72.841667        0.002439
Supercell matrix for 2nd force constants : 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


Inferring forces for displaced atoms and computing fcs ...


100%|██████████| 1/1 [00:00<00:00,  3.86it/s]


Max drift of force constants: 0.00020002 (yy) 0.00016579 (xx) 
Max drift after traditional symmetrization: -0.00000000 (zz) -0.00000000 (zz) 
Qpoints mesh for Brillouin Zone integration : [12 12 12]
Dispersion relations using phonopy for C8 ...

ph: [[2 0 0]
 [0 2 0]
 [0 0 2]]
Has imaginary phonon: False
Thermal properties written to /media/kai/data/NTE/software/qha/example/calc_mattersim/qha/phonon_5/thermal_properties.yaml and thermal_properties.dat
Supercell matrix for 2nd force constants : 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


Inferring forces for displaced atoms and computing fcs ...


100%|██████████| 1/1 [00:00<00:00,  3.57it/s]


Max drift of force constants: -0.00039221 (yy) -0.00034754 (xx) 
Max drift after traditional symmetrization: -0.00000000 (zz) -0.00000000 (zz) 
Qpoints mesh for Brillouin Zone integration : [12 12 12]
Dispersion relations using phonopy for C8 ...

ph: [[2 0 0]
 [0 2 0]
 [0 0 2]]
Has imaginary phonon: False
Thermal properties written to /media/kai/data/NTE/software/qha/example/calc_mattersim/qha/phonon_0/thermal_properties.yaml and thermal_properties.dat
Supercell matrix for 2nd force constants : 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


Inferring forces for displaced atoms and computing fcs ...


...

Max drift of force constants: 0.00008356 (yy) 0.00005139 (xx) 
Max drift after traditional symmetrization: 0.00000000 (xx) 0.00000000 (xx) 
Qpoints mesh for Brillouin Zone integration : [12 12 12]
Dispersion relations using phonopy for C8 ...

ph: [[2 0 0]
 [0 2 0]
 [0 0 2]]
Has imaginary phonon: False
Thermal properties written to /media/kai/data/NTE/software/qha/example/calc_mattersim/qha/phonon_8/thermal_properties.yaml and thermal_properties.dat
Supercell matrix for 2nd force constants : 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


Inferring forces for displaced atoms and computing fcs ...


100%|██████████| 1/1 [00:00<00:00,  3.83it/s]


Max drift of force constants: 0.00006877 (zx) 0.00000604 (xx) 
Max drift after traditional symmetrization: 0.00000000 (yy) 0.00000000 (yy) 
Qpoints mesh for Brillouin Zone integration : [12 12 12]
Dispersion relations using phonopy for C8 ...

ph: [[2 0 0]
 [0 2 0]
 [0 0 2]]
Has imaginary phonon: False
Thermal properties written to /media/kai/data/NTE/software/qha/example/calc_mattersim/qha/phonon_9/thermal_properties.yaml and thermal_properties.dat
Supercell matrix for 2nd force constants : 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


Inferring forces for displaced atoms and computing fcs ...


100%|██████████| 1/1 [00:00<00:00,  3.74it/s]


Max drift of force constants: 0.00025979 (yy) 0.00022893 (xx) 
Max drift after traditional symmetrization: -0.00000000 (yy) -0.00000000 (yy) 
Qpoints mesh for Brillouin Zone integration : [12 12 12]
Dispersion relations using phonopy for C8 ...

ph: [[2 0 0]
 [0 2 0]
 [0 0 2]]
Has imaginary phonon: False
Thermal properties written to /media/kai/data/NTE/software/qha/example/calc_mattersim/qha/phonon_10/thermal_properties.yaml and thermal_properties.dat
# Vinet EOS
#          T           E_0           B_0          B'_0           V_0
      0.000000    -72.484402    445.812066      3.762916     45.667392
     10.000000    -72.484402    445.812029      3.762945     45.667391
     20.000000    -72.484402    445.812058      3.762919     45.667392
     30.000000    -72.484402    445.812025      3.762929     45.667392
     40.000000    -72.484402    445.811978      3.762927     45.667392
     50.000000    -72.484404    445.811870      3.762928     45.667393
     60.000000    -72.484406    445.

In [ ]:
# 6. 热导率 (Leaf/Kappa)
# 使用 Phono3py 计算三阶力常数和晶格热导率
# 警告：计算量非常大，通常需要较长时间

kappa_set = KappaSet(calculator=calculator)
kappa_dir = "/media/kai/data/NTE/software/qha/example/calc_mattersim/kappa"

# dim_fc3: 三阶力常数超胞
# mesh:用于求解玻尔兹曼方程的网格

kappa_set.run_kappa(
    structure=relaxed_atoms,
    dim_fc3=np.array([2, 2, 2]),
    dim_fc2=np.array([2, 2, 2]),
    mesh=[11, 11, 11],
    work_dir=kappa_dir,
    temp_range=np.arange(0, 1001, 10) # 温度范围 0K 到 1000K 步长 10K
)
print(f"Kappa calculation finished in '{kappa_dir}'")


Initializing Phono3py for FC3...


Calculating FC3 Forces: 100%|██████████| 111/111 [00:30<00:00,  3.62it/s]


Calculating FC2...


Calculating FC2 Forces: 100%|██████████| 1/1 [00:00<00:00,  3.47it/s]


Calculating Thermal Conductivity...
Thermal conductivity calculation finished. Check kappa-m111111.hdf5
Reading file: kappa-m111111.hdf5
Plot saved to kappa_plot.png
Kappa calculation finished in '/media/kai/data/NTE/software/qha/example/calc_mattersim/kappa'
